# Log marginal likelihood

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gaussian_process.GPfunctions as gp
from gaussian_process import GaussianProcess, GPPrediction
from gaussian_process.kernels import SquaredExponentialKernel
import itertools

In [ ]:
# Objective function

objectiveFunction = lambda x: -x * np.sin(x)
objectiveFunctionDerivative = lambda x: -x * np.cos(x) - np.sin(x)

X = np.linspace(start=-3.0, stop=3.0, num=1_000)
y = objectiveFunction(X)
g = objectiveFunctionDerivative(X)

rng = np.random.default_rng(1)
training_indices = rng.choice(np.arange(y.size), size=40, replace=False)
X_train, y_train, g_train = (
    X[training_indices],
    y[training_indices],
    g[training_indices],
)

noise = 0.0

In [ ]:
def plot_gp(processes: list[GaussianProcess], label):
    fig, (ax1) = plt.subplots(1, 1)
    gp.plot_objective(ax1, X, y, X_train, y_train)

    for process in processes:
        pred = GPPrediction(X, process)
        gp.plot_gp(
            ax1,
            X,
            pred.mean,
            pred.std_deviation,
            label=f"l={process.kernel.l}",
        )

    gp.plot_label(ax1, label)
    plt.show()

In [ ]:
def catch(func, handle=lambda e: e, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except Exception as e:
        return handle(e)

In [ ]:
ls = [0.1, 0.5, 1.0, 1.5]

processes = list(
    filter(
        lambda x: x is not None,
        [
            catch(
                GaussianProcess,
                lambda e: None,
                kernel=SquaredExponentialKernel(l=l),
                x_known=X_train,
                f_known=y_train,
                f_noise=noise,
                g_noise=noise,
            )
            for l in ls
        ],
    )
)

plot_gp(processes, SquaredExponentialKernel.__name__)
for process in processes:
    try:
        print(
            f"Log marginal likelihood of {process.kernel.__class__.__name__} with l={process.kernel.l} without gradient info is {process.log_marginal_likelihood_stable()}"
        )
    except Exception:
        pass

processes = list(
    filter(
        lambda x: x is not None,
        [
            catch(
                GaussianProcess,
                lambda e: None,
                kernel=SquaredExponentialKernel(l=l),
                x_known=X_train,
                f_known=y_train,
                g_known=g_train,
                f_noise=noise,
                g_noise=noise,
            )
            for l in ls
        ],
    )
)

plot_gp(processes, SquaredExponentialKernel.__name__)
for process in processes:
    try:
        print(
            f"Log marginal likelihood of {process.kernel.__class__.__name__} with l={process.kernel.l} with gradient info is {process.log_marginal_likelihood()}"
        )
    except Exception:
        pass

## Plot of log marginal likelihood

In [ ]:
def get_likelihood(l):
    try:
        process = GaussianProcess(
            SquaredExponentialKernel(l=l), X_train, y_train, g_train, noise, noise
        )
        return process.log_marginal_likelihood()
    except Exception:
        return -np.inf


ls = np.linspace(start=1e-5, stop=1, num=1000)
likelihoods = [get_likelihood(l) for l in ls]

fig, (ax1) = plt.subplots(1, 1)
ax1.plot(ls, likelihoods)

ax1.grid()
ax1.set_title("Likelihood over hyperparameter l")
ax1.set(xlabel="$l$", ylabel="$log p(y|X)$")
plt.show()

best_l = ls[np.argmax(likelihoods)]
print(f"best l at {best_l} with {np.max(likelihoods)}")
plot_gp(
    [
        GaussianProcess(
            SquaredExponentialKernel(l=best_l), X_train, y_train, g_train, noise, noise
        )
    ],
    SquaredExponentialKernel.__name__,
)

## Using average distance as length scale

In [ ]:
average_distance = (np.max(X_train) - np.min(X_train)) / (
    2.0 * len(X_train)
)  # TODO: Think about if there is a smarter choice than 2
process = GaussianProcess(
    SquaredExponentialKernel(l=average_distance),
    X_train,
    y_train,
    g_train,
    noise,
    noise,
)

plot_gp([process], label=SquaredExponentialKernel.__name__)

print(
    f"log marginal likelihood at l={average_distance} is {process.log_marginal_likelihood()}"
)

## Using minimum distance as length scale

In [ ]:
min_distance = np.min([np.abs(a - b) for a, b in itertools.pairwise(X_train)])
process = GaussianProcess(
    SquaredExponentialKernel(l=min_distance), X_train, y_train, g_train, noise, noise
)

plot_gp([process], label=SquaredExponentialKernel.__name__)

print(
    f"log marginal likelihood at l={min_distance} is {process.log_marginal_likelihood()}"
)